Ce notebook montre comment préparer des données Open Food Facts pour entraîner un modèle évaluant la compatibilité d'une liste d'ingrédients.

Le bloc suivant extrait du fichier JSONL compressé un CSV `ingredients.csv` plus facile à manipuler.


In [ ]:
import gzip
import json
import csv

INPUT_FILE  = './data/openfoodfacts-products.jsonl.gz'
OUTPUT_FILE = './data/ingredients.csv'

with gzip.open(INPUT_FILE, 'rt', encoding='utf-8') as source, \
     open(OUTPUT_FILE, 'w', newline='', encoding='utf-8') as target:

    writer = csv.writer(target)
    writer.writerow(['code', 'ingredients'])

    for line in source:
        line = line.strip()
        if not line:
            continue
        try:
            product = json.loads(line)
        except json.JSONDecodeError:
            # ligne corrompue ou incomplète : on l'ignore
            continue

        # on préfère le texte français si dispo
        ing = product.get('ingredients_text_fr') or product.get('ingredients_text')
        if ing:
            writer.writerow([
                product.get('code', ''),
                ing.replace('\n', ' ').strip()
            ])



**Chargement et nettoyage** : on ouvre `ingredients.csv`, on met le texte en minuscules, on retire la ponctuation superflue puis on découpe les listes en tokens (colonne `tokens`).


In [ ]:
import pandas as pd
import re

# 1. Lire le CSV
df = pd.read_csv(
    './data/ingredients.csv',
    dtype={'ingredients': str},
    low_memory=False
)


df['ingredients'] = df['ingredients'].fillna('')

# 2. Nettoyage et tokenisation simple
def clean_and_tokenize(text):
    # minuscules, retirer ponctuation sauf ‘;’
    text = text.lower()
    text = re.sub(r'[^a-z0-9éèàçùœ \-;]', ' ', text)
    # split sur ‘;’ puis strip des blancs
    return [tok.strip() for tok in text.split(';') if tok.strip()]

df['tokens'] = df['ingredients'].apply(clean_and_tokenize)

print(df[['ingredients','tokens']].head())

**EntraÃ®nement d'un modÃ¨le Word2Vec** pour obtenir des vecteurs reprÃ©sentant chaque ingrÃ©dient Ã  partir des tokens.

**Optimisation** : on rÃ¨gle `workers=os.cpu_count()` pour exploiter tous les cÅurs du processeur. Gensim ne profite pas directement du GPU (RTXÂ 2050) mais la parallÃ©lisation CPU rÃ©duit nettement le temps d'Ã©ntraÃ®nement sur un i9.


In [ ]:
import os
from gensim.models import Word2Vec

sentences = df['tokens'].tolist()

num_workers = os.cpu_count()  # utilise tous les coeurs
w2v = Word2Vec(
    sentences,
    vector_size=100,
    window=5,
    min_count=5,
    sg=1,
    epochs=10,
    workers=num_workers,
)

vec_tomate = w2v.wv['tomate']


**Embedding moyen par produit** : on calcule la moyenne des vecteurs d'ingrédients pour chaque liste (`list_emb`).


In [ ]:
import numpy as np

def list_embedding(tokens, model):
    vecs = [model.wv[t] for t in tokens if t in model.wv]
    if not vecs:
        return np.zeros(model.vector_size)
    return np.mean(vecs, axis=0)

df['list_emb'] = df['tokens'].apply(lambda toks: list_embedding(toks, w2v))

**Score de compatibilité automatique** : on mesure la similarité moyenne entre toutes les paires d'ingrédients pour produire un score dans l'intervalle [0,1].


In [ ]:
def compatibility_score(tokens, model):
    pairs = []
    for i in range(len(tokens)):
        for j in range(i+1, len(tokens)):
            if tokens[i] in model.wv and tokens[j] in model.wv:
                pairs.append(model.wv.similarity(tokens[i], tokens[j]))
    if not pairs:
        return 0.5
    sim = float(np.mean(pairs))
    return (sim + 1) / 2

df['score'] = df['tokens'].apply(lambda toks: compatibility_score(toks, w2v))


**Préparation des données d'entraînement** : `X` regroupe les embeddings moyens et `y` contient les scores calculés précédemment.


In [ ]:
X = np.vstack(df['list_emb'].values)

y = df['score'].values


**Entraînement du réseau** : on définit `ScoringNet`, construit un DataLoader puis on entraîne le modèle quelques itérations. Si un GPU est disponible, le modèle et les batchs y sont transférés pour accélérer l'entraînement.


In [ ]:
import os
import torch, torch.nn as nn, torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.backends.cudnn.benchmark = True

class ScoringNet(nn.Module):
    def __init__(self, emb_dim=100):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(emb_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.net(x)

X_t = torch.tensor(X, dtype=torch.float32)
y_t = torch.tensor(y.reshape(-1, 1), dtype=torch.float32)
ds = TensorDataset(X_t, y_t)
loader = DataLoader(ds, batch_size=32, shuffle=True, num_workers=os.cpu_count(), pin_memory=(device.type=='cuda'))

model = ScoringNet(X.shape[1]).to(device)
opt = optim.AdamW(model.parameters(), lr=1e-3)
crit = nn.MSELoss()

for epoch in range(5):
    for xb, yb in loader:
        xb = xb.to(device, non_blocking=True)
        yb = yb.to(device, non_blocking=True)
        opt.zero_grad()
        pred = model(xb)
        loss = crit(pred, yb)
        loss.backward()
        opt.step()
    print(f'epoch {epoch} loss {loss.item():.4f}')
